# Calculate the metrics for test set

In [1]:
import os

In [19]:
niigz_files = os.listdir()
niigz_files.sort()
# niigz_files

groundtruth_files = []
predict_files = []
for niigz_file in niigz_files:
    if "groundtruth" in niigz_file:
        groundtruth_files.append(niigz_file)
    if "predict" in niigz_file:
        predict_files.append(niigz_file)

groundtruth_files.sort()
predict_files.sort()

case_names = []
for file in groundtruth_files:
    case_name = file.split("-groundtruth")[0]
    case_names.append(case_name)

case_names.sort()
# print(case_names)

test_dict = dict()
for case_name in case_names:
    test_dict[case_name] = {}
    groundtruth_niigz = case_name + "-groundtruth.nii.gz"
    predict_niigz = case_name + "-predict.nii.gz"
    test_dict[case_name]["groundtruth"] = groundtruth_niigz
    test_dict[case_name]["predict"] = predict_niigz

print(len(test_dict))

19


In [11]:
import SimpleITK as sitk
import numpy as np

def load_CT_scan_3D_image(niigz_file_name):
    itkimage = sitk.ReadImage(niigz_file_name)
    numpyImages = sitk.GetArrayFromImage(itkimage)
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
    return numpyImages, numpyOrigin, numpySpacing

def false_positive_rate_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    fp = np.sum(pred - pred * label) + smooth
    fpr = round(fp * 100 / (np.sum((1.0 - label)) + smooth), 3)
    return fpr

def false_negative_rate_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    fn = np.sum(label - pred * label) + smooth
    fnr = round(fn * 100 / (np.sum(label) + smooth), 3)
    return fnr

def sensitivity_calculation(pred, label):   #  identical to True-Positive-Rate
    sensitivity = round(100 - false_negative_rate_calculation(pred, label), 3)
    return sensitivity

def dice_coefficient_score_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    intersection = np.sum(pred * label)
    dice_coefficient_score = round(((2.0 * intersection + smooth) / (np.sum(pred) + np.sum(label) + smooth)) * 100, 2)
    return dice_coefficient_score

def precision_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    tp = np.sum(pred * label) + smooth
    precision = round(tp * 100 / (np.sum(pred) + smooth), 3)
    return precision


In [20]:
for item in test_dict.items():
    case_name = item[0]
    gt_npy, origin, spacing = load_CT_scan_3D_image(item[1]["groundtruth"])
    pred_npy, _, _ = load_CT_scan_3D_image(item[1]["predict"])
    
    FPR = false_positive_rate_calculation(pred_npy, gt_npy)
    FNR = false_negative_rate_calculation(pred_npy, gt_npy)
    Sensitivity = sensitivity_calculation(pred_npy, gt_npy)
    Precision = precision_calculation(pred_npy, gt_npy)
    DSC = dice_coefficient_score_calculation(pred_npy, gt_npy)
    
    print("{0} & {1} & {2} & {3} & {4} & {5}"
          .format(case_name, FPR, FNR, Sensitivity, Precision, DSC))
    

ATM_001_0000 & 0.006 & 7.24 & 92.76 & 98.089 & 95.35
ATM_024_0000 & 0.031 & 7.236 & 92.764 & 90.916 & 91.83
ATM_034_0000 & 0.022 & 3.705 & 96.295 & 96.251 & 96.27
ATM_041_0000 & 0.047 & 4.808 & 95.192 & 87.189 & 91.02
ATM_060_0000 & 0.024 & 2.534 & 97.466 & 93.12 & 95.24
ATM_061_0000 & 0.028 & 3.121 & 96.879 & 91.897 & 94.32
ATM_074_0000 & 0.053 & 3.838 & 96.162 & 87.038 & 91.37
ATM_075_0000 & 0.023 & 4.633 & 95.367 & 93.65 & 94.5
ATM_080_0000 & 0.026 & 4.067 & 95.933 & 93.183 & 94.54
ATM_150_0000 & 0.082 & 3.902 & 96.098 & 80.036 & 87.33
ATM_158_0000 & 0.044 & 3.461 & 96.539 & 86.953 & 91.5
ATM_163_0000 & 0.035 & 4.301 & 95.699 & 91.958 & 93.79
ATM_197_0000 & 0.032 & 9.234 & 90.766 & 90.81 & 90.79
ATM_245_0000 & 0.038 & 0.309 & 99.691 & 83.496 & 90.88
ATM_246_0000 & 0.04 & 0.611 & 99.389 & 82.853 & 90.37
ATM_260_0000 & 0.016 & 1.041 & 98.959 & 94.227 & 96.53
ATM_266_0000 & 0.019 & 2.351 & 97.649 & 93.112 & 95.33
ATM_271_0000 & 0.049 & 1.044 & 98.956 & 86.544 & 92.33
ATM_638_0000 & 0.0